voici la mission que je vous propose pour cette après-midi : 

Nous souhaitons utiliser le clustering (et dont Kmeans) pour analyser les données texte annoté (émotions) en notre possessions et identifier les documents anormaux et obtenir des piste pour améliorer les résultats de classification.

En commençant par le premier jeux de donnée (le plus petit)

1. Est-que les clusters obtenue avec kmeans reflete les groupements de texte par émotions ?
2. comparer les résultats de plusieurs modéles dans un tableaux avec le temps, l'inertie, la NMI, la AMI, l'ARI Index et la silouhette.
3. quelle est le nombre de cluster optimal aux regards de la métrique "silouhette" ? de l'inertie ?  est ce que l'inertie est corrélé à la mesure de silouhette ?
4. reproduire ces expériences avec le jeux de donnée 2.
5. Peut-on identifier des cluster de documents ou des émotion difficile à classifier ? (ie cluster non homogéne)

Je vous propose le point à 13h30 sur la mission, l'objectif est que chacun arrive à allerjusqu"au point 3.

Bon appétit

In [72]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv("datasets/Emotion_final.csv")
#df2 = pd.read_csv("datasets/text_emotion.csv")

In [73]:
print("----------------------------")
print("First lines of the dataframe")
print("----------------------------")
df.head()

----------------------------
First lines of the dataframe
----------------------------


,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [74]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

targets = df['Emotion']
corpus = df['Text']
vect = CountVectorizer()
X = vect.fit_transform(corpus)
words = vect.get_feature_names()

In [75]:
from sklearn.preprocessing import LabelEncoder # to use centered matrix
le = LabelEncoder()                            #
y = le.fit_transform(targets)                  #

In [76]:
from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = make_pipeline(StandardScaler(with_mean=False), kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(data, estimator[-1].labels_,
                                 metric="euclidean")
    ]

    # Show the results
    formatter_result = ("{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}"
                        "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}")
    print(formatter_result.format(*results))

## 1. Est-que les clusters obtenue avec kmeans reflete les groupements de texte par émotions ?

### As we can see in the previous notebook kmeans has no brain and makes calculations like a machine without no text comprehension :)

## 2. comparer les résultats de plusieurs modéles dans un tableaux avec le temps, l'inertie, la NMI, la AMI, l'ARI Index et la silouhette.
### Pipeline to control multiple results

In [77]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

print(82 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')

kmeans = KMeans(init="k-means++", n_clusters=6, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="k-means++", data=X, labels=y)

kmeans = KMeans(init="random", n_clusters=6, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=X, labels=y)

clf = TruncatedSVD()
Xpca = clf.fit_transform(X)
XX = PCA(2).fit_transform(Xpca)
kmeans = KMeans(n_clusters=6, n_init=1)
bench_k_means(kmeans=kmeans, name="PCA-based", data=XX, labels=y)
from sklearn.feature_extraction.text import TfidfVectorizer
print(82 * '_')

__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette
k-means++	0.934s	411698579	0.000	0.064	0.001	0.000	0.000	-0.024
random   	0.594s	411963612	0.000	0.073	0.001	-0.000	0.000	-0.080
PCA-based	0.201s	9257	0.003	0.003	0.003	0.003	0.003	0.403
__________________________________________________________________________________


## Improved results with TF-IDF

In [83]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

print(82 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')

kmeans = KMeans(init="k-means++", n_clusters=6, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="k-means++", data=X, labels=y)

kmeans = KMeans(init="random", n_clusters=6, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=X, labels=y)

from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer()
tf_idf = tf_idf_vectorizer.fit_transform(corpus)

clf = TruncatedSVD()
Xpca = clf.fit_transform(tf_idf)
XX = PCA(2).fit_transform(Xpca)
kmeans = KMeans(n_clusters=6, n_init=1)
bench_k_means(kmeans=kmeans, name="PCA-based", data=XX, labels=y)
from sklearn.feature_extraction.text import TfidfVectorizer
print(82 * '_')

__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette
k-means++	0.681s	411698579	0.000	0.064	0.001	0.000	0.000	-0.024
random   	0.681s	411963612	0.000	0.073	0.001	-0.000	0.000	-0.080
PCA-based	0.093s	8319	0.003	0.003	0.003	0.003	0.003	0.360
__________________________________________________________________________________


## We can say when Inertia is big, silhouette is null or very very low
## Also, the reciprocity is truth when silhouette has a value, inertia tends to a value < 10000